In [ ]:
using DataFrames
using Compat
using DataFramesMeta
using ProgressMeter
using PyPlot

In [ ]:
include("../fund.jl/src/fund.jl")
include("../fund.jl/src/marginaldamages.jl")

In [ ]:
rice_regions = ["US", "EU", "Japan", "Russia", "Eurasia", "China", "India", "MidEast", "Africa", "LatAm", "OHI", "OthAsia"]
fund_regions = ["USA", "CAN", "WEU", "JPK", "ANZ", "EEU", "FSU", "MDE", "CAM", "LAM", "SAS", "SEA", "CHI", "MAF", "SSA", "SIS"];

Allocate some Dicts that hold the output per model. Key is model name

In [ ]:
σ_squareds = @compat Dict{String,Matrix{Float64}}()
cpcs = @compat Dict{String,Matrix{Float64}}()
pops = @compat Dict{String,Matrix{Float64}}()
globpops = @compat Dict{String,Vector{Float64}}()
mds = @compat Dict{String,Matrix{Float64}}()

# FUND

In [ ]:
# Price inflator from $1995 to $2014, taken from http://www.bls.gov/data/inflation_calculator.htm
const priceInflatorFUND = 1.55

## Get $\sigma^2$ for FUND

In [ ]:
x1 = readcsv("data/fund_sigmasquared.csv")
# Fix column/row layout
x2 = x1'
# Original uses a 5 year timestep, this repeats each year entry five times to get to a yearly matrix
x3 = repeat(x2, inner=[5,1])
# Create a matrix for later years that are not covered in the input file, by simply repeating the last year
x4 = repmat(x3[end,:], 195, 1)
# Combine the matrix for the covered years with the matrix for the later years
x5 = vcat(x3, x4)
# Done
σ_squareds["FUND"] = x5[6:end,:];

## Run FUND

In [ ]:
param_fund = loadparameters("../fund.jl/data");
m_fund = getfund(params=param_fund);
run(m_fund)
pops["FUND"] = m_fund[:population, :populationin1][65:65+284,:];
globpops["FUND"] = vec(sum(m_fund[:population, :populationin1], 2))[65:65+284]
cpcs["FUND"] = (m_fund[:socioeconomic, :consumption] ./ m_fund[:population, :populationin1])[65:65+284,:] .* priceInflatorFUND
mds["FUND"] = getmarginaldamages(parameters=param_fund, emissionyear=2015)[65:65+284,:].*12/44;

/# RICE

In [ ]:
# Price inflator from $2005 to $2014, taken from http://www.bls.gov/data/inflation_calculator.htm
const priceInflatorRICE = 1.21

## Get $\sigma^2$ for RICE

In [ ]:
x1 = readcsv("data/rice_sigmasquared.csv")
# Fix column/row layout
x2 = x1'
# Original uses a 5 year timestep, this repeats each year entry five times to get to a yearly matrix
x3 = repeat(x2, inner=[5,1])
# Create a matrix for later years that are not covered in the input file, by simply repeating the last year
x4 = repmat(x3[end,:], 195, 1)
# Combine the matrix for the covered years with the matrix for the later years
x5 = vcat(x3, x4)
# Done
σ_squareds["RICE"] = x5[6:end,:];

## Run RICE

In [ ]:
x1 = readcsv("data/rice_md.csv")
# Fix column/row layout
x2 = x1'
# Original uses a 10 year timestep, this repeats each year entry ten times to get to a yearly matrix
x3 = repeat(x2, inner=[10,1])
# Leave out some later years to match σ_squared size
x4 = x3[1:285,:]
# Price level adjustment
x5 = x4 .* priceInflatorRICE
mds["RICE"] = x5.*12/44;

In [ ]:
x1 = readcsv("data/rice_cpc.csv")
# Fix column/row layout
x2 = x1'
# Original uses a 10 year timestep, this repeats each year entry ten times to get to a yearly matrix
x3 = repeat(x2, inner=[10,1])
# Leave out some later years to match σ_squared size
x4 = x3[1:285,:]
# Price level adjustment
x5 = x4 .* priceInflatorRICE
cpcs["RICE"] = x5;

In [ ]:
x1 = readcsv("data/rice_population.csv")
# Fix column/row layout
x2 = x1'
# Original uses a 10 year timestep, this repeats each year entry ten times to get to a yearly matrix
x3 = repeat(x2, inner=[10,1])
# Leave out some later years to match σ_squared size
x4 = x3[1:285,:]
pops["RICE"] = x4
globpops["RICE"] = vec(sum(x4, 2));

# Output for JE

In [ ]:
writecsv("output/RICE_sigmasquared.csv", σ_squareds["RICE"])
writecsv("output/FUND_sigmasquared.csv", σ_squareds["FUND"])
writecsv("output/RICE_cpc.csv", cpcs["RICE"])
writecsv("output/FUND_cpc.csv", cpcs["FUND"])
writecsv("output/RICE_pop.csv", pops["RICE"])
writecsv("output/FUND_pop.csv", pops["FUND"]);

# Compute SCC

Main equation of the paper

In [ ]:
function equation11(cpc, pop, globpop, md, σsquared, σZero, γ, η, ρ, α, focus_region, I, c_ede)
    timesteps, regions = size(cpc)
    
    for t=1:timesteps, r=1:regions
        if σZero
            I[t, r] = 1.0 - exp(-0.5 * γ * 0.0)
        else
            I[t, r] = 1.0 - exp(-0.5 * γ * σsquared[t, r])
        end
    end
      
    for t=1:timesteps
        if γ == 1.
            # DOUBLE CHECK
            temp_x = 0.
            for r=1:regions
                temp_x += pop[t,r] * log(cpc[t,r] * (1.0 - I[t,r]))
            end
            c_ede[t] = exp( temp_x / globpop[t])
        else
            temp_x = 0.
            for r=1:regions
                temp_x += pop[t,r] * (cpc[t,r] * (1.0 - I[t,r]))^(1.0 - γ)
            end
            c_ede[t] =(temp_x/globpop[t])^(1.0 / (1.0 - γ))
        end
    end

    globc = 0.
    for r=1:regions
        globc += pop[1,r] * cpc[1,r]
    end
    
    cpc0x = focus_region == -1 ? globc / globpop[1] : cpc[1, focus_region]

    scc = 0.
    for t=1:timesteps, r=1:regions
        scc +=  md[t, r] *                      # P_rt * d_rt in the paper
                (1.0 + ρ)^(-(t-1)) *          # time discount factor
                (cpc0x / cpc[t, r])^γ *         # Normal equity weight
                (c_ede[t] / c_ede[1])^(γ - η) * # New thing
                (1.0 - I[t, r])^(-(γ + 1.0)) *  # New thing 2
                (1.0 - I[t, r])^(2 * α)         # This is Δ
    end
    return scc
end;

Modify this to run for different parameter combinations.

In [ ]:
σZeros = [true, false]
γs = linspace(0., 2., 21)
ηs = linspace(0., 2., 21)
ρs = [0.001, 0.015, 0.03]
#ρs = linspace(0., 0.06, 13)
#αs = [0., 1.];
αs = linspace(0., 1.5, 16);

Compute SCCs

In [ ]:
df = DataFrame([Float64, Float64, Float64, Float64, Int64, Bool, Float64, String], [:rho, :eta, :gamma, :alpha, :focusregion, :sigmazero, :scc, :model], 0)

for model=["RICE", "FUND"]
    cpc = cpcs[model]
    pop = pops[model]
    globpop = globpops[model]
    md = mds[model]
    σSquared = σ_squareds[model]
    
    # Pre-allocate some arrays
    c_ede = similar(globpops[model])
    I = similar(mds[model]);    
    
    focus_regions = vcat(-1, collect(1:size(cpc,2)))
    
    progress = Progress(length(σZeros) * length(γs) * length(ηs) * length(ρs) * length(αs) * length(focus_regions),1,"Computing $model...")

    for σZero=σZeros, γ=γs, η=ηs, ρ=ρs, α=αs, focus_region=focus_regions
        scc = equation11(cpc, pop, globpop, md, σSquared, σZero, γ, η, ρ, α, focus_region, I, c_ede)
        push!(df, (ρ,η,γ,α,focus_region,σZero,scc,model))
        next!(progress)
    end
end

# Plotting

## Figure 1

In [ ]:
df_temp = @where(df, (:rho .== 0.015) & (:focusregion .== 1) & (:alpha .== 1.) & (:gamma .<= 1.5) )

df_figure1_panelA_line1 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== true) & (:eta .== :gamma) )
df_figure1_panelA_line2 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== true) & (:eta .== 1.5) )

df_figure1_panelB_line1 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== true) & (:eta .== :gamma) )
df_figure1_panelB_line2 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== true) & (:eta .== 1.5) )
nothing

In [ ]:
f, axs = subplots(1,2, sharex= true, sharey=false, figsize=(8,4))
f[:subplots_adjust](bottom=0.15)

axs[1][:plot](df_figure1_panelA_line1[:gamma], df_figure1_panelA_line1[:scc])
axs[1][:plot](df_figure1_panelA_line2[:gamma], df_figure1_panelA_line2[:scc])
axs[1][:set_title]("FUND")
axs[1][:legend]([L"$\eta=\gamma$", L"$\eta=1.5$"], "upper left")
axs[1][:set_xlim]([0., 1.5])
axs[1][:grid]()
axs[1][:set_xlabel](L"$\gamma$")
axs[1][:set_ylabel](L"$\mathrm{\$/tCO_{2}}$")

axs[2][:plot](df_figure1_panelB_line1[:gamma], df_figure1_panelB_line1[:scc])
axs[2][:plot](df_figure1_panelB_line2[:gamma], df_figure1_panelB_line2[:scc])
axs[2][:set_title]("RICE")
axs[2][:grid]()
axs[2][:set_xlabel](L"$\gamma$")

f[:suptitle](L"\mathrm{Social\/Cost\/of\/Carbon\/(}\rho=0.015,\/\mathrm{US\/normalization)}")

f[:subplots_adjust](top=0.85)

savefig("output/SCC_by_gamma.pdf")

nothing

## Figure 2

In [ ]:
df_temp = @where(df, (:rho .== 0.015) & (:focusregion .== 1) & (:gamma .== 0.7) & (:eta .== 1.5) )

df_figure2_panelA_line1 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== true) )
df_figure2_panelA_line2 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== false) )
df_figure2_panelA_line3 = @where(df, (:rho .== 0.015) & (:focusregion .== 1) & (:gamma .== 0.) & (:eta .== 1.5) & (:model .== "FUND") & (:sigmazero .== false) )

df_figure2_panelB_line1 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== true) )
df_figure2_panelB_line2 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== false) )
df_figure2_panelB_line3 = @where(df, (:rho .== 0.015) & (:focusregion .== 1) & (:gamma .== 0.) & (:eta .== 1.5) & (:model .== "RICE") & (:sigmazero .== false) )
nothing

In [ ]:
f, axs = subplots(1,2, sharex= true, sharey=false, figsize=(8,4))
f[:subplots_adjust](bottom=0.15)

axs[1][:plot](df_figure2_panelA_line1[:alpha], df_figure2_panelA_line1[:scc])
axs[1][:plot](df_figure2_panelA_line2[:alpha], df_figure2_panelA_line2[:scc])
axs[1][:plot](df_figure2_panelA_line3[:alpha], df_figure2_panelA_line3[:scc])
axs[1][:set_title]("FUND")
axs[1][:legend](["regional inequality", "country inequality", "no inequality"], "middle center")
axs[1][:set_xlim]([0., 1.5])
axs[1][:set_ylim]([0., 10])
axs[1][:grid]()
axs[1][:set_xlabel](L"$\alpha$")
axs[1][:set_ylabel](L"$\mathrm{\$/tCO_{2}}$")

axs[2][:plot](df_figure2_panelB_line1[:alpha], df_figure2_panelB_line1[:scc])
axs[2][:plot](df_figure2_panelB_line2[:alpha], df_figure2_panelB_line2[:scc])
axs[2][:plot](df_figure2_panelB_line3[:alpha], df_figure2_panelB_line3[:scc])
axs[2][:set_title]("RICE")
axs[2][:set_ylim]([0., 50])
axs[2][:grid]()
axs[2][:set_xlabel](L"$\alpha$")

f[:suptitle](L"\mathrm{Social\/Cost\/of\/Carbon\/(}\rho=0.015,\/\eta=1.5,\/\gamma=0.7,\/\mathrm{US\/normalization)}")

f[:subplots_adjust](top=0.85)

savefig("output/SCC_by_alpha.pdf")

nothing

## Figure 3

In [ ]:
df_temp = @where(df, (:rho .== 0.015) & (:eta .== 1.5) & (:sigmazero .== true) & (:alpha .== 0.) & (:focusregion .!= -1) )

df_figure3_panelA_line1 = @where(df_temp, (:model .== "FUND") & (:gamma .== 0.0))
sort!(df_figure3_panelA_line1, cols=[:scc])

df_figure3_panelA_line2= @where(df_temp, (:model .== "FUND") & (:gamma .== 0.1))
sort!(df_figure3_panelA_line2, cols=[:scc])

df_figure3_panelA_line3= @where(df_temp, (:model .== "FUND") & (:gamma .== 0.3))
sort!(df_figure3_panelA_line3, cols=[:scc])

df_figure3_panelA_line4= @where(df_temp, (:model .== "FUND") & (:gamma .== 0.5))
sort!(df_figure3_panelA_line4, cols=[:scc])

df_figure3_panelA_line5= @where(df_temp, (:model .== "FUND") & (:gamma .== 0.7))
sort!(df_figure3_panelA_line5, cols=[:scc])

df_figure3_panelB_line1 = @where(df_temp, (:model .== "RICE") & (:gamma .== 0.0))
sort!(df_figure3_panelB_line1, cols=[:scc])

df_figure3_panelB_line2 = @where(df_temp, (:model .== "RICE") & (:gamma .== 0.1))
sort!(df_figure3_panelB_line2, cols=[:scc])

df_figure3_panelB_line3 = @where(df_temp, (:model .== "RICE") & (:gamma .== 0.3))
sort!(df_figure3_panelB_line3, cols=[:scc])

df_figure3_panelB_line4 = @where(df_temp, (:model .== "RICE") & (:gamma .== 0.5))
sort!(df_figure3_panelB_line4, cols=[:scc])

df_figure3_panelB_line5 = @where(df_temp, (:model .== "RICE") & (:gamma .== 0.7))
sort!(df_figure3_panelB_line5, cols=[:scc])

nothing

In [ ]:
f, axs = subplots(1,2, sharex= false, sharey=false, figsize=(8,4))
f[:subplots_adjust](bottom=0.25)

axs[1][:plot](collect(1:16), df_figure3_panelA_line1[:scc], marker="o")
axs[1][:plot](collect(1:16), df_figure3_panelA_line2[:scc], marker="o")
axs[1][:plot](collect(1:16), df_figure3_panelA_line3[:scc], marker="o")
axs[1][:plot](collect(1:16), df_figure3_panelA_line4[:scc], marker="o")
axs[1][:plot](collect(1:16), df_figure3_panelA_line5[:scc], marker="o")
axs[1][:set_xticks](collect(1:16))
axs[1][:set_xticklabels]([fund_regions[i] for i in df_figure3_panelA_line2[:focusregion]], rotation="vertical", ha="center")
axs[1][:set_title]("FUND")
axs[1][:set_xlabel]("normalization")
axs[1][:set_ylabel](L"$\mathrm{\$/tCO_{2}}$")
axs[1][:legend]([L"\gamma=0.0", L"\gamma=0.1", L"\gamma=0.3", L"\gamma=0.5", L"\gamma=0.7"], "upper left")

axs[2][:plot](collect(1:12), df_figure3_panelB_line1[:scc], marker="o")
axs[2][:plot](collect(1:12), df_figure3_panelB_line2[:scc], marker="o")
axs[2][:plot](collect(1:12), df_figure3_panelB_line3[:scc], marker="o")
axs[2][:plot](collect(1:12), df_figure3_panelB_line4[:scc], marker="o")
axs[2][:plot](collect(1:12), df_figure3_panelB_line5[:scc], marker="o")
axs[2][:set_xticks](collect(1:12))
axs[2][:set_xticklabels]([rice_regions[i] for i in df_figure3_panelB_line2[:focusregion]], rotation="vertical", ha="center")
axs[2][:set_title]("RICE")
axs[2][:set_xlabel]("normalization")

f[:suptitle](L"\mathrm{Social\/Cost\/of\/Carbon\/(}\rho=0.015,\/\eta=1.5,\/\mathrm{region\/inequality)}")

f[:subplots_adjust](top=0.85)

savefig("output/SCC_by_normalization.pdf")

nothing

# Figure 4

In [ ]:
df_temp = @where(df, (:rho .== 0.015) & ( ((:focusregion .== 9) & (:model.=="RICE")) | ((:focusregion .==15) & (:model.=="FUND")) ) & (:alpha .== 1.) & (:gamma .<= 1.5) )

df_figure4_panelA_line1 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== true) & (:eta .== :gamma) )
df_figure4_panelA_line2 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== true) & (:eta .== 1.5) )

df_figure4_panelB_line1 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== true) & (:eta .== :gamma) )
df_figure4_panelB_line2 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== true) & (:eta .== 1.5) )
nothing

In [ ]:
f, axs = subplots(1,2, sharex= true, sharey=false, figsize=(8,4))
f[:subplots_adjust](bottom=0.15)

axs[1][:plot](df_figure4_panelA_line1[:gamma], df_figure4_panelA_line1[:scc])
axs[1][:plot](df_figure4_panelA_line2[:gamma], df_figure4_panelA_line2[:scc])
axs[1][:set_title]("FUND")
axs[1][:legend]([L"$\eta=\gamma$", L"$\eta=1.5$"], "upper left")
axs[1][:set_xlim]([0., 1.5])
axs[1][:grid]()
axs[1][:set_xlabel](L"$\gamma$")
axs[1][:set_ylabel](L"$\mathrm{\$/tCO_{2}}$")

axs[2][:plot](df_figure4_panelB_line1[:gamma], df_figure4_panelB_line1[:scc])
axs[2][:plot](df_figure4_panelB_line2[:gamma], df_figure4_panelB_line2[:scc])
axs[2][:set_title]("RICE")
axs[2][:grid]()
axs[2][:set_xlabel](L"$\gamma$")

f[:suptitle](L"\mathrm{Social\/Cost\/of\/Carbon\/(}\rho=0.015,\/\mathrm{poor\/normalization)}")

f[:subplots_adjust](top=0.85)

savefig("output/SCC_by_gamma_for_poor_normalization.pdf")

nothing

## Table rho sensitivity

In [ ]:
df_table3 = @where(df, (:focusregion .== 1) & (:alpha .== 1.) & (:gamma .== 0.7) & (:eta .== 1.5) & (:sigmazero .== true) )
df_table3 = DataFrame(model=df_table3[:model], rho=df_table3[:rho], scc=df_table3[:scc])
df_table3 = unstack(df_table3, :rho, :model, :scc)
rename!(df_table3, :model, :rho)
sort!(df_table3, cols=[:rho])
df_table3

In [ ]:
df_table3b = @where(df, (:focusregion .== 1) & (:alpha .== 1.) & (:gamma .== 0.0) & (:eta .== 1.5) & (:sigmazero .== true) )
df_table3b = DataFrame(model=df_table3b[:model], rho=df_table3b[:rho], scc=df_table3b[:scc])
df_table3b = unstack(df_table3b, :rho, :model, :scc)
rename!(df_table3b, :model, :rho)
sort!(df_table3b, cols=[:rho])
df_table3b

## Table central results

In [ ]:
df_table4 = DataFrame(Any, 0, 4)
push!(df_table4, ["", "parameter specification", "FUND", "RICE"])
model_base_fund = @where(df, (:model.=="FUND") & (:rho.==0.015) & (:eta.==1.5) & (:gamma.==0.) & (:alpha.==1.) & (:sigmazero.==true) & (:focusregion.==1))[1,:scc]
model_base_rice = @where(df, (:model.=="RICE") & (:rho.==0.015) & (:eta.==1.5) & (:gamma.==0.) & (:alpha.==1.) & (:sigmazero.==true) & (:focusregion.==1))[1,:scc]
push!(df_table4, ["model base value", "eta=1.5, rho=0.015", model_base_fund, model_base_rice])
model_ri_fund = @where(df, (:model.=="FUND") & (:rho.==0.015) & (:eta.==1.5) & (:gamma.==0.7) & (:alpha.==1.) & (:sigmazero.==true) & (:focusregion.==1))[1,:scc]
model_ri_rice = @where(df, (:model.=="RICE") & (:rho.==0.015) & (:eta.==1.5) & (:gamma.==0.7) & (:alpha.==1.) & (:sigmazero.==true) & (:focusregion.==1))[1,:scc]
push!(df_table4, ["disentangled inequality aversion", "and gamma=0.7", model_ri_fund, model_ri_rice])
model_ci_fund = @where(df, (:model.=="FUND") & (:rho.==0.015) & (:eta.==1.5) & (:gamma.==0.7) & (:alpha.==1.) & (:sigmazero.==false) & (:focusregion.==1))[1,:scc]
model_ci_rice = @where(df, (:model.=="RICE") & (:rho.==0.015) & (:eta.==1.5) & (:gamma.==0.7) & (:alpha.==1.) & (:sigmazero.==false) & (:focusregion.==1))[1,:scc]
push!(df_table4, ["and country inequality", "and alpha=1.0", model_ci_fund, model_ci_rice])
df_table4